In [17]:
import pandas as pd
import numpy as np

In [18]:
def get_frame():
    df = pd.read_excel(io='BountyTime.xlsx',sheet_name='BountyMats')
    df_lookup = pd.read_excel(io='BountyTime.xlsx',sheet_name='MatLevels')
    df_combined = pd.merge(df,df_lookup,on='Difficulty')
    df_combined['Duration'] = df_combined['Time'].apply(lambda x: pd.Timedelta(str(x))).astype('timedelta64[s]')
    df_combined['MatRate'] = df_combined['Mats'] / df_combined['Duration']
    df_combined['DiffClass'] = df_combined['Class'] + '-' + df_combined['Difficulty']
    return df_combined[['Class','DiffClass', 'MatRate']]

In [19]:
import math
def get_ucb(row):
    cnt = max(row['count'],1)
    div1 = 2 * math.log(row['total']) / cnt
    ucb = math.sqrt(div1)
    return ucb + row['median']

In [20]:
def df_bandit_class(df):
    dfgb = df.groupby(['Class'])
    dfgb = dfgb['Quality'].agg(['count','median', 'var']).reset_index()
    dfgb['total'] = dfgb['count'].sum()
    dfgb['donext'] = dfgb.apply(get_ucb, axis=1).fillna(999)
    foo = dfgb.sort_values('donext', ascending=False)
    return foo

In [21]:
def df_bandit_diff(df):
    dfgb = df.groupby(['DiffClass'])
    dfgb = dfgb['Quality'].agg(['count','median', 'var']).reset_index()
    dfgb['total'] = dfgb['count'].sum()
    dfgb['donext'] = dfgb.apply(get_ucb, axis=1).fillna(999)
    foo = dfgb.sort_values('donext', ascending=False)
    return foo

In [22]:
def get_quality(df_num):
    df_med_dev = df_num.transform(lambda x: abs(x - x.median())).transform(lambda x: (x*0) + x.median() * 1.4826)
    df_mscore = df_num.transform(lambda x: x - x.median()) / df_med_dev
    df_quality = df_mscore.transform(lambda x: 1 / (1 + np.exp(-x))).to_frame('Quality')
    return df_quality

In [23]:
df = get_frame()

In [24]:
df_quality = get_quality(df['MatRate'])

In [25]:
df = df.join(df_quality).sort_values('Quality', ascending=False)

In [26]:
df.tail(10)

,Class,DiffClass,MatRate,Quality
40,Necro,Necro-T10,NaN,NaN
42,DemonHunter,DemonHunter-T11,NaN,NaN
43,Necro,Necro-T11,NaN,NaN
44,WitchDoctor,WitchDoctor-T11,NaN,NaN
45,DemonHunter,DemonHunter-T12,NaN,NaN
46,Necro,Necro-T12,NaN,NaN
47,WitchDoctor,WitchDoctor-T12,NaN,NaN
48,DemonHunter,DemonHunter-T13,NaN,NaN
49,Necro,Necro-T13,NaN,NaN
50,WitchDoctor,WitchDoctor-T13,NaN,NaN


In [27]:
df = df.sort_values('Quality', ascending=False)
df.head(10)

,Class,DiffClass,MatRate,Quality
32,WitchDoctor,WitchDoctor-T7,0.017058,0.858629
38,WitchDoctor,WitchDoctor-T9,0.016985,0.856693
35,WitchDoctor,WitchDoctor-T8,0.013559,0.738520
41,WitchDoctor,WitchDoctor-T10,0.012594,0.695743
14,WitchDoctor,WitchDoctor-T1,0.012000,0.667523
19,Necro,Necro-T3,0.011561,0.645851
17,WitchDoctor,WitchDoctor-T2,0.010830,0.608496
26,WitchDoctor,WitchDoctor-T5,0.010753,0.604441
18,DemonHunter,DemonHunter-T3,0.010714,0.602430
25,Necro,Necro-T5,0.010000,0.564455


In [28]:
df_class = df_bandit_class(df)

In [29]:
df_class

,Class,count,median,var,total,donext
1,Necro,9,0.511331,0.023689,30,1.380711
0,DemonHunter,7,0.302958,0.018440,30,1.288742
2,WitchDoctor,14,0.577009,0.041010,30,1.274063


In [30]:
nextClass = df_class.iloc[0,0]

In [31]:
df_filtered = df[df['Class'] == nextClass]

In [32]:
df_bandit_diff(df_filtered)

,DiffClass,count,median,var,total,donext
8,Necro-T13,0,NaN,NaN,9,999.000000
6,Necro-T11,0,NaN,NaN,9,999.000000
15,Necro-T8,0,NaN,NaN,9,999.000000
14,Necro-T7,0,NaN,NaN,9,999.000000
13,Necro-T6,0,NaN,NaN,9,999.000000
7,Necro-T12,0,NaN,NaN,9,999.000000
16,Necro-T9,0,NaN,NaN,9,999.000000
5,Necro-T10,0,NaN,NaN,9,999.000000
10,Necro-T3,1,0.645851,NaN,9,2.742145
12,Necro-T5,1,0.564455,NaN,9,2.660749
